In [1]:
import numpy as np
import os
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
# import seaborn as sns
import pickle
import time
import gc
from tqdm import tqdm, tqdm_notebook

%matplotlib inline

#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pylab import rcParams
rcParams['figure.figsize'] = 14, 6

import warnings
warnings.filterwarnings("ignore")

#中文字体
import matplotlib
matplotlib.use('qt4agg')
#指定默认字体
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['font.family'] = 'sans-serif'
#解决负号'-'显示为方块的问题
matplotlib.rcParams['axes.unicode_minus'] = False

In [2]:
# df = pd.read_csv('./feature/df_feature4_ctr_extra.csv', encoding='utf-8')
# df = df.drop(['prefix', 'query_prediction', 'title', 'title_tag', 'query_prediction_origin'], axis=1)

# cont_fea = ['dict_len', 'title_rate', 'max_title_rate', 'title_rate_max_sub', 'title_rank_in_query', 'title_rank_in_query_origin', 'rank_first_origin_rate', 'rank_first_origin_rate_sub']
# cont_fea += [i for i in df.columns if '_count' in i]
# cate_fea = ['is_max_rate', 'is_first_rate', 'title_equal_prefix', 'prefix_in_title', 'title_startswith_prefix','title_endswith_prefix', 'rate_big5', 'title_in_query']
# long_cate_fea = ['user_id', 'item_id']
# df.shape

(2100000, 118)

In [2]:
df = pd.read_csv('./feature/df_feature4_ctr_extra.csv', encoding='utf-8', nrows=1)

cont_fea = ['dict_len', 'title_rate', 'max_title_rate', 'title_rate_max_sub', 'title_rank_in_query', 'title_rank_in_query_origin', 'rank_first_origin_rate', 'rank_first_origin_rate_sub']
cont_fea += [i for i in df.columns if '_count' in i]
cate_fea = ['is_max_rate', 'is_first_rate', 'title_equal_prefix', 'prefix_in_title', 'title_startswith_prefix','title_endswith_prefix', 'rate_big5', 'title_in_query']
long_cate_fea = ['user_id', 'item_id']

df = pd.read_csv('./feature/df_feature4_ctr_extra.csv', encoding='utf-8', usecols=long_cate_fea+['tag', 'label', 'is_val']+cont_fea+cate_fea)

feature1 = pd.read_csv('./feature/feature_rank_query.csv')
feature2 = pd.read_csv('./feature/feature_rank_query_re.csv')
feature3 = pd.read_csv('./feature/feature_dict_dis.csv')
feature4 = pd.read_csv('./feature/feature_vector_dis.csv')
feature5 = pd.read_csv('./feature/feature_re_pretit_rate.csv')
feature6 = pd.read_csv('./feature/feature_titpre_dis.csv')
feature7 = pd.read_csv('./feature/feature_vector_dis_maxpool.csv')

cont_fea += list(feature1.columns)
cont_fea += list(feature2.columns)
cont_fea += list(feature3.columns)
cont_fea += list(feature4.columns)
cont_fea += list(feature5.columns)
cont_fea += list(feature6.columns)
cont_fea += list(feature7.columns)

df = pd.concat([df, feature1, feature2, feature3, feature4, feature5, feature6, feature7], axis=1)
df.shape

(2100000, 97)

In [3]:
df2 = pd.read_csv('./feature/featurefull.csv', encoding='gbk')

feature1 = pd.read_csv('./feature/featurefull_dis1.csv')
feature2 = pd.read_csv('./feature/featurefull_dis2.csv')
feature3 = pd.read_csv('./feature/featurefull_maxpool.csv')

df2 = pd.concat([df2, feature1, feature2, feature3], axis=1)
df2.shape

(2100000, 97)

In [5]:
col = ['label','tag','user_id','item_id','title_query_dict_cos_distance','title_max_distance','title_rank_in_query_origin','title_max_cos_distance', 'title_max_distance_maxpool', 'title_first_distance_maxpool','title_max_cos_distance_maxpool', 'title_first_cos_distance_maxpool','title_str_distance_maxpool', 'title_str_cos_distance_maxpool', 'title_rank_in_query_origin_count','title_rank_in_query_origin_tag_count','title_rate','title_in_query_count','title_in_query','title_in_query_tag_count','title_rank_in_query','rank_first_origin_rate_sub','title_prefix_cos_distance','title_rate_max_sub','title_rank_in_query_count','title_rank_in_query_tag_count','title_prefix_distance','title_str_cos_distance','title_str_distance','title_origin_str_cos_distance','title_origin_str_distance','is_max_rate_tag_count','is_max_rate_count','is_max_rate','title_equal_prefix_tag_count','title_equal_prefix_count','title_equal_prefix','prefix_inter_title_url_len_rate_title_url','prefix_inter_title_len_rate_title','title_first_distance','prefix_tag_count','title_query_dict_distance','title_startswith_prefix_tag_count','title_startswith_prefix_count','title_startswith_prefix','title_first_cos_distance','title_query_str_cos_distance','title_query_str_distance','rate_big5_tag_count','rate_big5_count','rate_big5','title_len','title_endswith_prefix_tag_count','prefix_in_title_count','prefix_in_title','prefix_in_title_tag_count','prefix_inter_title_len','prefix_inter_title_url_len','is_first_rate_count','is_first_rate','title_has_num','dict_len_tag_count','is_first_rate_tag_count','title_url_has_num','title_has_symbol','prefix_inter_title_len_rate_prefix','prefix_inter_title_url_len_rate_prefix','tag_rank_query_re']
col = list(set(col) & set(df2.columns))
len(col)

63

In [6]:
df = df.iloc[:2000000, :][col]
df2 = df2[col]
df = pd.concat([df, df2], axis=0)
df.shape

(4100000, 63)

## data

In [7]:
user_id_dict = dict(zip(df.user_id.unique(), range(len(df.user_id.unique()))))
item_id_dict = dict(zip(df.item_id.unique(), range(len(df.item_id.unique()))))

user_id_len = len(df.user_id.unique())
item_id_len = len(df.item_id.unique())

In [8]:
tag = pd.get_dummies(df.tag, prefix='tag')
cate_fea += list(tag.columns)
df = pd.concat([df, tag], axis=1)
df = df.drop(['tag'], axis=1)

df.shape

(4100000, 84)

In [9]:
cont_fea = list(set(cont_fea) & set(df.columns))

In [10]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
for fea in tqdm_notebook(cont_fea):
    scaler_val = df[fea][~df[fea].isnull()].values
    scaler = MinMaxScaler().fit(scaler_val.reshape((len(scaler_val), 1)))
    df[fea].fillna(df[fea].mean(), inplace=True)
    df[fea] = scaler.transform(df[fea].values.reshape((len(df), 1))).reshape((len(df),)).tolist()

A Jupyter Widget

In [12]:
cate_fea = list(set(cate_fea) & set(df.columns))

In [13]:
df.label = df.label.fillna(-1)
df.label = df.label.astype(int)
df[cate_fea] = df[cate_fea].astype(int)

In [14]:
good_fea = list(tag.columns)

# 0.6 22
# good_fea += ['title_query_dict_cos_distance','title_max_distance','title_rank_in_query_origin','title_max_cos_distance','title_rank_in_query_origin_count','title_rank_in_query_origin_tag_count','title_rate','title_in_query_count','title_in_query','title_in_query_tag_count','title_rank_in_query','rank_first_origin_rate_sub','title_rate_max_sub','title_rank_in_query_count','title_rank_in_query_tag_count','title_str_cos_distance','title_str_distance','title_origin_str_cos_distance','title_origin_str_distance','is_max_rate_tag_count','is_max_rate_count','is_max_rate']
# 0.59 25
# good_fea += ['title_query_dict_cos_distance','title_max_distance','title_rank_in_query_origin','title_max_cos_distance','title_rank_in_query_origin_count','title_rank_in_query_origin_tag_count','title_rate','title_in_query_count','title_in_query','title_in_query_tag_count','title_rank_in_query','rank_first_origin_rate_sub','title_rate_max_sub','title_rank_in_query_count','title_rank_in_query_tag_count','title_str_cos_distance','title_str_distance','title_origin_str_cos_distance','title_origin_str_distance','is_max_rate_tag_count','is_max_rate_count','is_max_rate','title_equal_prefix_tag_count','title_equal_prefix_count','title_equal_prefix']
# 0.58 27
# good_fea += ['title_query_dict_cos_distance','title_max_distance','title_rank_in_query_origin','title_max_cos_distance','title_rank_in_query_origin_count','title_rank_in_query_origin_tag_count','title_rate','title_in_query_count','title_in_query','title_in_query_tag_count','title_rank_in_query','rank_first_origin_rate_sub','title_rate_max_sub','title_rank_in_query_count','title_rank_in_query_tag_count','title_str_cos_distance','title_str_distance','title_origin_str_cos_distance','title_origin_str_distance','is_max_rate_tag_count','is_max_rate_count','is_max_rate','title_equal_prefix_tag_count','title_equal_prefix_count','title_equal_prefix','title_first_distance','prefix_tag_count']
# 0.57 32
# good_fea += ['title_query_dict_cos_distance','title_max_distance','title_rank_in_query_origin','title_max_cos_distance','title_rank_in_query_origin_count','title_rank_in_query_origin_tag_count','title_rate','title_in_query_count','title_in_query','title_in_query_tag_count','title_rank_in_query','rank_first_origin_rate_sub','title_rate_max_sub','title_rank_in_query_count','title_rank_in_query_tag_count','title_str_cos_distance','title_str_distance','title_origin_str_cos_distance','title_origin_str_distance','is_max_rate_tag_count','is_max_rate_count','is_max_rate','title_equal_prefix_tag_count','title_equal_prefix_count','title_equal_prefix','title_first_distance','prefix_tag_count','title_query_dict_distance','title_startswith_prefix_tag_count','title_startswith_prefix_count','title_startswith_prefix','title_first_cos_distance']

# 0.56 42
# good_fea += ['title_query_dict_cos_distance','title_max_distance','title_rank_in_query_origin','title_max_cos_distance','title_rank_in_query_origin_count','title_rank_in_query_origin_tag_count','title_rate','title_in_query_count','title_in_query','title_in_query_tag_count','title_rank_in_query','rank_first_origin_rate_sub','title_rate_max_sub','title_rank_in_query_count','title_rank_in_query_tag_count','title_str_cos_distance','title_str_distance','title_origin_str_cos_distance','title_origin_str_distance','is_max_rate_tag_count','is_max_rate_count','is_max_rate','title_equal_prefix_tag_count','title_equal_prefix_count','title_equal_prefix','prefix_inter_title_url_len_rate_title_url','prefix_inter_title_len_rate_title','title_first_distance','prefix_tag_count','title_query_dict_distance','title_startswith_prefix_tag_count','title_startswith_prefix_count','title_startswith_prefix','title_first_cos_distance','rate_big5_tag_count','rate_big5_count','rate_big5','title_len','title_endswith_prefix_tag_count','prefix_in_title_count','prefix_in_title','prefix_in_title_tag_count']
# 0.55 58
good_fea += ['title_query_dict_cos_distance','title_max_distance','title_rank_in_query_origin','title_max_cos_distance', 'title_max_distance_maxpool', 'title_first_distance_maxpool','title_max_cos_distance_maxpool', 'title_first_cos_distance_maxpool','title_str_distance_maxpool', 'title_str_cos_distance_maxpool', 'title_rank_in_query_origin_count','title_rank_in_query_origin_tag_count','title_rate','title_in_query_count','title_in_query','title_in_query_tag_count','title_rank_in_query','rank_first_origin_rate_sub','title_prefix_cos_distance','title_rate_max_sub','title_rank_in_query_count','title_rank_in_query_tag_count','title_prefix_distance','title_str_cos_distance','title_str_distance','title_origin_str_cos_distance','title_origin_str_distance','is_max_rate_tag_count','is_max_rate_count','is_max_rate','title_equal_prefix_tag_count','title_equal_prefix_count','title_equal_prefix','prefix_inter_title_url_len_rate_title_url','prefix_inter_title_len_rate_title','title_first_distance','prefix_tag_count','title_query_dict_distance','title_startswith_prefix_tag_count','title_startswith_prefix_count','title_startswith_prefix','title_first_cos_distance','title_query_str_cos_distance','title_query_str_distance','rate_big5_tag_count','rate_big5_count','rate_big5','title_len','title_endswith_prefix_tag_count','prefix_in_title_count','prefix_in_title','prefix_in_title_tag_count','prefix_inter_title_len','prefix_inter_title_url_len','is_first_rate_count','is_first_rate','title_has_num','dict_len_tag_count','is_first_rate_tag_count','title_url_has_num','title_has_symbol','prefix_inter_title_len_rate_prefix','prefix_inter_title_url_len_rate_prefix','tag_rank_query_re']

In [15]:
good_fea = list(set(good_fea) & set(df.columns))

In [8]:
cate_fea_dict = {'user_id':user_id_len, 'item_id':item_id_len}

with open('./feature/ffm_train(origin_label_goodfea55_double).txt', 'w') as f:
    for index, row in tqdm_notebook(df.iloc[:4000000, :].iterrows()):
        line = ''
        offset = 0
        order = 3
        
        line += str(row.label) + ' '
        line += '1:' + str(user_id_dict[row.user_id]+offset) + ':1 '
        offset += user_id_len
        line += '2:' + str(item_id_dict[row.item_id]+offset) + ':1 '
        offset += item_id_len
        
        for i in good_fea:
            if row[i] != 0:
                line += str(order) + ':' + str(offset) + ':' + str(row[i]) + ' '
            offset += 1
            order += 1
        
        line += '\n'
        _ = f.write(line)

A Jupyter Widget

In [9]:
cate_fea_dict = {'user_id':user_id_len, 'item_id':item_id_len}

with open('./feature/ffm_val(goodfea55_double).txt', 'w') as f:
    for index, row in tqdm_notebook(df.iloc[4000000:4050000, :].iterrows()):
        line = ''
        offset = 0
        order = 3
        
        line += str(row.label) + ' '
        line += '1:' + str(user_id_dict[row.user_id]+offset) + ':1 '
        offset += user_id_len
        line += '2:' + str(item_id_dict[row.item_id]+offset) + ':1 '
        offset += item_id_len
        
        for i in good_fea:
            if row[i] != 0:
                line += str(order) + ':' + str(offset) + ':' + str(row[i]) + ' '
            offset += 1
            order += 1
        
        line += '\n'
        _ = f.write(line)

A Jupyter Widget

In [10]:
cate_fea_dict = {'user_id':user_id_len, 'item_id':item_id_len}

with open('./feature/ffm_test(goodfea55_double).txt', 'w') as f:
    for index, row in tqdm_notebook(df.iloc[4050000:410000, :].iterrows()):
        line = ''
        offset = 0
        order = 3
        
        line += '1:' + str(user_id_dict[row.user_id]+offset) + ':1 '
        offset += user_id_len
        line += '2:' + str(item_id_dict[row.item_id]+offset) + ':1 '
        offset += item_id_len
        
        for i in good_fea:
            if row[i] != 0:
                line += str(order) + ':' + str(offset) + ':' + str(row[i]) + ' '
            offset += 1
            order += 1
        
        line += '\n'
        _ = f.write(line)

A Jupyter Widget

## FFM

In [5]:
import xlearn as xl
from sklearn.metrics import f1_score

In [6]:
len(df.user_id.unique())
len(df.item_id.unique())

167198

278384